In [2]:
# Import necessary libraries
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os

In [3]:
import dagshub

mlflow.set_tracking_uri("https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow")
dagshub.init(repo_owner='sumitwatkar', repo_name='Text-Classification-using-MLOPS', mlflow=True)

Accessing as sumitwatkar

Initialized MLflow to track repo "sumitwatkar/Text-Classification-using-MLOPS"

Repository sumitwatkar/Text-Classification-using-MLOPS initialized!

In [4]:
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [5]:
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [6]:
# Normalize the text data
df = normalize_text(df)

In [7]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

C:\Users\watka\AppData\Local\Temp\ipykernel_10816\1508851655.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


In [9]:
# Set the experiment name
mlflow.set_experiment("BOW vs TFIDF")

2024/08/09 23:34:01 INFO mlflow.tracking.fluent: Experiment with name 'BOW vs TFIDF' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/06265aecc5484f85a96a1ccb812e4a9b', creation_time=1723226657582, experiment_id='2', last_update_time=1723226657582, lifecycle_stage='active', name='BOW vs TFIDF', tags={}>

In [10]:
# Define feature extraction methods
vectorizers = {
    'BoW': CountVectorizer(),
    'TF-IDF': TfidfVectorizer()
}

In [11]:
# Define algorithms
algorithms = {
    'LogisticRegression': LogisticRegression(),
    'MultinomialNB': MultinomialNB(),
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier()
}

In [13]:
 # Start the parent run
with mlflow.start_run(run_name="All Experiments") as parent_run:
    # Loop through algorithms and feature extraction methods (Child Runs)
    for algo_name, algorithm in algorithms.items():
        for vec_name, vectorizer in vectorizers.items():
            with mlflow.start_run(run_name=f"{algo_name} with {vec_name}", nested=True) as child_run:
                X = vectorizer.fit_transform(df['content'])
                y = df['sentiment']
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                

                # Log preprocessing parameters
                mlflow.log_param("vectorizer", vec_name)
                mlflow.log_param("algorithm", algo_name)
                mlflow.log_param("test_size", 0.2)
                
                # Model training
                model = algorithm
                model.fit(X_train, y_train)
                
                # Log model parameters
                if algo_name == 'LogisticRegression':
                    mlflow.log_param("C", model.C)
                elif algo_name == 'MultinomialNB':
                    mlflow.log_param("alpha", model.alpha)
                elif algo_name == 'XGBoost':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                elif algo_name == 'RandomForest':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("max_depth", model.max_depth)
                elif algo_name == 'GradientBoosting':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                    mlflow.log_param("max_depth", model.max_depth)
                
                # Model evaluation
                y_pred = model.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)
                
                # Log evaluation metrics
                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("precision", precision)
                mlflow.log_metric("recall", recall)
                mlflow.log_metric("f1_score", f1)
                
                # Log model
                mlflow.sklearn.log_model(model, "model")
                
                # Save and log the notebook
                notebook_path = "exp2_bow_vs_tfidf.ipynb"
                os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
                mlflow.log_artifact(notebook_path)

                # Print the results for verification
                print(f"Algorithm: {algo_name}, Feature Engineering: {vec_name}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1}")


c:\Users\watka\anaconda3\envs\myenv\lib\site-packages\_distutils_hack\__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\watka\anaconda3\envs\myenv\lib\site-packages\_distutils_hack\__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2024/08/09 23:41:49 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is no

Algorithm: LogisticRegression, Feature Engineering: BoW
Accuracy: 0.7946987951807228
Precision: 0.7862001943634597
Recall: 0.7970443349753694
F1 Score: 0.7915851272015656


2024/08/09 23:41:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run LogisticRegression with BoW at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/2/runs/5f7588ef3330453898b3cca3927ce877.
2024/08/09 23:41:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/2.
c:\Users\watka\anaconda3\envs\myenv\lib\site-packages\_distutils_hack\__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\watka\anaconda3\envs\myenv\lib\site-packages\_distutils_hack\__init__.py:3

Algorithm: LogisticRegression, Feature Engineering: TF-IDF
Accuracy: 0.7956626506024096
Precision: 0.7795648060548723
Recall: 0.8118226600985222
F1 Score: 0.7953667953667953


2024/08/09 23:42:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run LogisticRegression with TF-IDF at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/2/runs/ec455deba73149618e85774ccc6dffc7.
2024/08/09 23:42:17 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/2.
c:\Users\watka\anaconda3\envs\myenv\lib\site-packages\_distutils_hack\__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\watka\anaconda3\envs\myenv\lib\site-packages\_distutils_hack\__init__.p

...

2024/08/09 23:46:34 INFO mlflow.tracking._tracking_service.client: 🏃 View run RandomForest with TF-IDF at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/2/runs/6abc80084ece4d568ad62f1e1c1b80af.
2024/08/09 23:46:34 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/2.
c:\Users\watka\anaconda3\envs\myenv\lib\site-packages\_distutils_hack\__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\watka\anaconda3\envs\myenv\lib\site-packages\_distutils_hack\__init__.py:32: 

Algorithm: GradientBoosting, Feature Engineering: BoW
Accuracy: 0.727710843373494
Precision: 0.8016085790884718
Recall: 0.5891625615763547
F1 Score: 0.6791595684270301


2024/08/09 23:47:16 INFO mlflow.tracking._tracking_service.client: 🏃 View run GradientBoosting with BoW at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/2/runs/f091e3e4ce7d47c3ac7a0681226e0d76.
2024/08/09 23:47:16 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/2.
c:\Users\watka\anaconda3\envs\myenv\lib\site-packages\_distutils_hack\__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\watka\anaconda3\envs\myenv\lib\site-packages\_distutils_hack\__init__.py:32:

Algorithm: GradientBoosting, Feature Engineering: TF-IDF
Accuracy: 0.7243373493975903
Precision: 0.8030095759233926
Recall: 0.5783251231527093
F1 Score: 0.6723940435280642


2024/08/09 23:47:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run GradientBoosting with TF-IDF at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/2/runs/99a198d5869f4f42a52fb9c6e7e3461d.
2024/08/09 23:47:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/2.
2024/08/09 23:47:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run All Experiments at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/2/runs/458fee69bfd945719a62f2254d4a1adf.
2024/08/09 23:47:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/2.
